## **Load libraries and data**

In [85]:
# !pip install --upgrade pythainlp
# !pip install pyLDAvis
# !pip install -U pandas-profiling
# !pip install sefr_cut

     |████████████████████████████████| 8.7MB 6.1MB/s 
     |████████████████████████████████| 327kB 36.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85400 sha256=567ebe6096eafe7e52dad756343dbda3869bba4b93e1af59f7264f128d4cf202
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [308]:
import pandas as pd
import pythainlp
import sefr_cut
import gensim
# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [309]:
df = pd.read_csv('CustomerReviews2.csv')

In [310]:
df.head(5)

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,สิ่งที่เราคิด
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,"พนักงาน,อาหารหลากหลาย,เนื้อ,น้ำจิ้ม"
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,"คุณภาพ,ชาบู,บริการ,เครื่องดื่ม"
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,"โต๊ะ,น้ำจิ้ม,ชาบูน้ำดำ,น้ำซุบเห็ด,เนื้อสัตว์,ช..."
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,"ชาบู,ราคา,เนื้อ,โปร,เนื้ออส,ไอศกรีม"
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,"ชาบู,โปร,บัตรเครดิต,สาขา,ที่นั่ง,เด็ก,คุณภาพ,ขนม"


## **Tokenize Words**

In [311]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n','\n\n', 'ร้าน', '(', ')' , '           ','–', '!', '!!','-','/','+','😆','🤣','"','','%','\u200b','::']
screening_words = stopwords + removed_words

sefr_cut.load_model(engine='ws1000')
def tokenize_with_space(sentence):
  merged = ''
  # words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  words = sefr_cut.tokenize(sentence)
  for word in words[0]:
    if word not in screening_words:
      word = word.rstrip("\n")
      word = word.rstrip("\u200b")
      if word is None or word == ' ' or word == '' or word ==':':
        continue
      else: 
        merged = merged + ',' + word
    elif word is None:
      continue
  return merged[1:]
  
  # return words

loading model.....
Success


In [312]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [313]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Review ID         21 non-null     int64  
 1   Restaurant_ID     21 non-null     object 
 2   Restaurant        21 non-null     object 
 3   User              21 non-null     object 
 4   Headline          21 non-null     object 
 5   Review            21 non-null     object 
 6   Rating            20 non-null     float64
 7   สิ่งที่เราคิด     19 non-null     object 
 8   Review_tokenized  21 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.6+ KB


In [314]:
df['Review_tokenized'].iloc[10]

'อัพเดทราคา,ชาบูชิ,ตอน,399,บาท,net,ทาน,1.,15,ชม.,น้ำซุป,เลือก,4,รส,ซูชิลาย,ตา,เทมปุระ,ทาน,เพลิน,ดี,ยืน,ทาน,ไลน์,คาว,บุฟเฟ่ต์,กุ้ง,สด,เบ,เนื้อ,สไลด์,หลัก,อิ่ม,คุ้ม'

In [315]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,สิ่งที่เราคิด,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"สาขา,ยูเนี่ยนมอลล์,เดอะมอลล์บางกะปิ,ค่ายมวย,พิ...","ลอง,สาขา,ยู,เนี่ยน,มอลล์,รอบ,สาขา,เดอะมอลล์,บา..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟต์,โปรโมชัน,น้ำ,VAT,วันธรรมดา,ชอบ,เวลา,น้...","บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,รวมVAT,ทาน,..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"union mall,ชาบู,ไม่เคยได้ลองกิน,พิซซ่า,สปาเก็ต...","รู้จัก,union,mall,ลอง,กิน,ขึ้นใจ,ชื่อ,ชื่อ,อยู..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,NaN,"อร่อย,คุ้ม,ค่า,สมราคา,บุฟเฟ่,หมู,ผัก,น้ำ,จบ,25..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,NaN,"จ่าย,ราคา,199,บาท,จ่าย,เงิน,สด,ราคา,น้ำ,VAT 7,..."


## **Create Dictionary**

In [316]:
# documents = df['Review_tokenized'].to_list()
documents = df['Review_tokenized']
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [317]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ก็รสชาติ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'จิ้ม', 'ซุป', 'ดี', 'ติม', 'ต้องการ', 'ถาด', 'ถาม', 'น้ำ', 'บริการ', 'บริการรสชาติ', 'บาท', 'ประทับใจ', 'พนักงาน', 'รสชาติ', 'รัก', 'ราคา', 'ร่อย', 'ละกัน', 'สรุป', 'สะอาด', 'สะอ้าน', 'สัน', 'อร่อย', 'อาหาร', 'เคลม', 'เจี้ยว', 'เซ็ท', 'เต้า', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แปลก', 'แฟน', 'โมจิอ', 'ใจ', 'ใส่', 'ไอ', '\u200b ', '\u200bกับ', '\u200bดำ', '\u200bดี', '\u200bพอ', '\u200bภาพ', '\u200bหลากหลาย', '\u200bแต่', '\u200bใหม่', '\u200bไม่', '21', '30', '9', 'CTW', 'Terminal', 'การงดรับ', 'งาน', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ดื่ม', 'ทะยอย', 'ทาน', 'ทำ', 'นาที', 'นึก', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'รายละเอียด', 'ลูกค้า', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่อง', 'เค้า', 'เจอ', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอกาส', 'โอเค', '', '2', '555', '600+', 'กก', 'กรุบ', 'กั้น', 'คน', 'คาว', 'คุโรบู', 'ค่ะ', 'ค่า', 'จาน', 'จืด', 'ฉาก', 'ชอบ', 'ชา', 'ชาบูน้ำดำ', 'ชิ้น', 'ชื่อ', 'ซอยซอส', 'ตก', 'ตะ', 'ตัก', 'ตัว',

In [318]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [528]:
num_topics = 5
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 100
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 395 ms, sys: 1.49 ms, total: 397 ms
Wall time: 397 ms


In [529]:
# pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.052827 -0.019655       1        1  44.157173
2     -0.043791 -0.031769       2        1  22.700896
0      0.004426  0.017427       3        1  13.218764
4     -0.013396  0.032089       4        1  12.519259
3     -0.000065  0.001907       5        1   7.403907, topic_info=       Term       Freq      Total Category  logprob  loglift
68      ทาน  24.000000  24.000000  Default  30.0000  30.0000
11       ดี  29.000000  29.000000  Default  29.0000  29.0000
145   เลือก  19.000000  19.000000  Default  28.0000  28.0000
32    อาหาร  21.000000  21.000000  Default  27.0000  27.0000
37    เนื้อ  24.000000  24.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
102     จาน   0.478863   4.092157   Topic5  -5.2925   0.4577
3       กิน   0.759508  21.662645   Topic5  -4.8312  -0.7475
132      สด   0.651873  13.362118   Topic5  -4.9840  -0.4172
97       คน   0.644263  15.502810   Topic5  -4.9958  -0.5775
22   รสชาติ   0.567293  10.600515   Topic5  -5.1230  -0.3246

[342 rows x 6 columns], token_table=      Topic      Freq Term
term                      
90        2  0.819761     
202       3  0.432360    )
202       5  0.432360    )
0         1  0.702710    1
0         2  0.175678    1
...     ...       ...  ...
45        3  0.166718   ไอ
45        4  0.166718   ไอ
45        5  0.166718   ไอ
250       1  0.921283  ไอศ
46        1  0.738219   ​ 

[329 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 5, 4])

#Review Topic

In [547]:
list_topic = list()
for i in range(0,num_topics):
  print('topic:',i)
  list_topic.append(set(convert_to_topic(model.show_topic(i,20))))
  print(list_topic[i])

topic: 0
{'เลือก', 'เนื้อ', 'อาหาร', 'ราคา', 'ซูชิ', 'กุ้ง', 'จิ้ม', 'บาท', 'รสชาติ', 'เมนู', 'สด', 'คุ้ม', 'คน', 'น้ำ', 'ทาน', 'ดี', 'เทมปุระ', 'บุฟเฟ่ต์', 'สาขา', 'อร่อย'}
topic: 1
{'เวลา', 'เลือก', 'เนื้อ', 'อาหาร', 'ซุป', 'ราคา', 'กุ้ง', 'กิน', 'บาท', 'รสชาติ', 'สด', 'คุ้ม', 'คน', 'น้ำ', 'ทาน', 'ดี', 'หวาน', 'ตัว', 'สาขา', 'อร่อย'}
topic: 2
{'', 'เลือก', 'เนื้อ', 'อาหาร', 'ชอบ', 'ซุป', 'ราคา', 'กิน', 'ไก่', 'รสชาติ', 'สด', 'คุ้ม', 'น้ำ', 'ทาน', 'ดี', 'หวาน', 'หมู', 'ลอง', 'อร่อย', 'สั่ง'}
topic: 3
{'สาย', 'เลือก', 'เนื้อ', 'อาหาร', 'หน้า', 'ราคา', 'กิน', 'หม้อ', 'ติม', 'ค่า', 'บาท', 'คุ้ม', 'น้ำ', '3', 'ทาน', 'ดี', 'หวาน', 'ตัว', 'สาขา', 'อร่อย'}
topic: 4
{'เลือก', 'เนื้อ', 'อาหาร', 'ชอบ', 'ราคา', 'พนักงาน', 'กิน', 'ชาบู', 'บริการ', 'คุ้ม', 'คุณภาพ', 'น้ำ', 'นึก', 'ทาน', 'ดี', 'นั่ง', 'หวาน', 'สาขา', 'อร่อย', 'สั่ง'}


# Find unique Keyword each topic

In [531]:
c = list()
for i in range(0,num_topics):
  a = list_topic[i]
  b = set()
  for j in range(0,num_topics):
    if i != j:
      b = b.union(list_topic[j])
  a = a-b
  c.append(list(a))
  print(i,a)

0 {'เมนู', 'เทมปุระ', 'ซูชิ', 'บุฟเฟ่ต์', 'จิ้ม'}
1 {'เวลา'}
2 {'', 'หมู', 'ไก่', 'ลอง'}
3 {'สาย', 'หม้อ', 'ติม', 'หน้า', 'ค่า', '3'}
4 {'ชาบู', 'บริการ', 'คุณภาพ', 'พนักงาน', 'นึก', 'นั่ง'}


# Find  Keyword Intersection for each topic

In [548]:
d = list_topic[0]
print(d)
for i in range(1,num_topics):
  # print(list_topic[i])
  d = d.intersection(list_topic[i])
  print(i,d)

{'เลือก', 'เนื้อ', 'อาหาร', 'ราคา', 'ซูชิ', 'กุ้ง', 'จิ้ม', 'บาท', 'รสชาติ', 'เมนู', 'สด', 'คุ้ม', 'คน', 'น้ำ', 'ทาน', 'ดี', 'เทมปุระ', 'บุฟเฟ่ต์', 'สาขา', 'อร่อย'}
1 {'เลือก', 'เนื้อ', 'อาหาร', 'สด', 'อร่อย', 'คุ้ม', 'คน', 'บาท', 'น้ำ', 'ราคา', 'รสชาติ', 'ทาน', 'กุ้ง', 'สาขา', 'ดี'}
2 {'เลือก', 'เนื้อ', 'อาหาร', 'สด', 'คุ้ม', 'ดี', 'น้ำ', 'ราคา', 'รสชาติ', 'ทาน', 'อร่อย'}
3 {'เลือก', 'เนื้อ', 'อาหาร', 'อร่อย', 'คุ้ม', 'น้ำ', 'ราคา', 'ทาน', 'ดี'}
4 {'เลือก', 'เนื้อ', 'อาหาร', 'คุ้ม', 'ดี', 'น้ำ', 'ราคา', 'ทาน', 'อร่อย'}


In [534]:
def convert_to_keyword(x):
  new_x = str(c[x])
  return new_x


In [535]:
def convert_to_topic(x):
  new_x = [i[0] for i in x]
  return new_x


In [536]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])
df['topic_name'] = df['Review_tokenized'].apply(lambda x: model.show_topic(model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0]))
df['topic_name2'] = df['topic_name'].apply(lambda x:convert_to_topic(x))
df['keyword'] = df['topics'].apply(lambda x:convert_to_keyword(x))
# df2 = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(','))))

In [537]:
df[['Restaurant', 'Review', 'topics','score','topic_name2','keyword']]

,Restaurant,Review,topics,score,topic_name2,keyword
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,1,0.998467,"[กิน, ดี, น้ำ, อาหาร, คน, เนื้อ, ทาน, ราคา, บา...",['เวลา']
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,4,0.997341,"[ดี, เนื้อ, ทาน, อร่อย, สาขา, น้ำ, เลือก, ราคา...","['ชาบู', 'บริการ', 'คุณภาพ', 'พนักงาน', 'นึก',..."
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,2,0.999215,"[เนื้อ, น้ำ, อร่อย, ทาน, เลือก, , ดี, ราคา, อา...","['', 'หมู', 'ไก่', 'ลอง']"
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4,0.995929,"[ดี, เนื้อ, ทาน, อร่อย, สาขา, น้ำ, เลือก, ราคา...","['ชาบู', 'บริการ', 'คุณภาพ', 'พนักงาน', 'นึก',..."
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,4,0.996865,"[ดี, เนื้อ, ทาน, อร่อย, สาขา, น้ำ, เลือก, ราคา...","['ชาบู', 'บริการ', 'คุณภาพ', 'พนักงาน', 'นึก',..."
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,2,0.996423,"[เนื้อ, น้ำ, อร่อย, ทาน, เลือก, , ดี, ราคา, อา...","['', 'หมู', 'ไก่', 'ลอง']"
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,0,0.997597,"[ทาน, อาหาร, ดี, น้ำ, เลือก, เนื้อ, กุ้ง, ราคา...","['เมนู', 'เทมปุระ', 'ซูชิ', 'บุฟเฟ่ต์', 'จิ้ม']"
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,1,0.997976,"[กิน, ดี, น้ำ, อาหาร, คน, เนื้อ, ทาน, ราคา, บา...",['เวลา']
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,1,0.996851,"[กิน, ดี, น้ำ, อาหาร, คน, เนื้อ, ทาน, ราคา, บา...",['เวลา']
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,0,0.997447,"[ทาน, อาหาร, ดี, น้ำ, เลือก, เนื้อ, กุ้ง, ราคา...","['เมนู', 'เทมปุระ', 'ซูชิ', 'บุฟเฟ่ต์', 'จิ้ม']"


In [401]:
df[[ 'Review', 'topics','score','topic_name2','keyword']].to_csv('out.csv', index=False)

#Result

จากการทดสอบพบว่า Topic Name สามารถแบ่งได้เป็น 5 Topic จากที่ดูจาก LDA Bubble chart พบว่า สามารถแบ่งกลุ่ม ได้แยกจากกันมากที่ ซึ่งโดย
ทุกๆ Topic มี Keyword ร่วมกัน โดยจากการทำ intersect topic name กับทุกๆ Topic คือ

'เลือก', 'เนื้อ', 'อาหาร', 'คุ้ม', 'ดี', 'น้ำ', 'ราคา', 'ทาน', 'อร่อย'
ซึ่งในแต่ละ Review โดยส่วนใหญ่ มีพูดถึง เนื้อ อาหาร น้ำซุป ราคาอาหาร กิน ดี มีของหวาน
แต่ก็จะมีบาง Topic จะมีเอกลักษณ์ที่พูดถึงแตกต่างกันไป ซึ่งดูจาก Different keywork ของแต่ละ Topic
* Topic 0 พูดถึง เมนูเทมปุระ ซูชิ น้ำจิ้ม (accurancy : 55%)
* Topic 1 พูดถึง เวลา (accurancy : 28%)
* Topic 2 พูดถึง หมู ไก่ ลองกิน (accurancy : 70%)
* Topic 3 พูดถึง สาย หม้อ ไอติม (accurancy : 100%)
* Topic 4 พูดถึง ชาบู บริการ คุณภาพ พนักงาน (accurancy : 46%)

ดังนั้นแต่ละ Topic ควรชื่อว่า
* Topic 0 คุ้มดีเนื้ออร่อย มีเทมปุระ ซูชิ และน้ำจิ้ม
* Topic 1 คุ้มดีเนื้ออร่อย มีเวลา
* Topic 2 คุ้มดีเนื้ออร่อย มีลองกินหมูไก่
* Topic 3 คุ้มดีเนื้ออร่อย มีหม้อมีไอติม
* Topic 4 คุ้มดีชาบูอร่อย พนักงานบริการคุณภาพ


In [549]:
#Evaluate
for i in c:
  g = list()
  for j in list(i):
    # print(i,j)
    g.append(df[df["keyword"].str.contains(j)&df["Review"].str.contains(j)]['Review'].count()/df[df["keyword"].str.contains(j)]['keyword'].count())
  print(i,"\naccurancy:",sum(g)/len(g))


['เมนู', 'เทมปุระ', 'ซูชิ', 'บุฟเฟ่ต์', 'จิ้ม'] 
accurancy: 0.55
['เวลา'] 
accurancy: 0.2857142857142857
['', 'หมู', 'ไก่', 'ลอง'] 
accurancy: 0.7
['สาย', 'หม้อ', 'ติม', 'หน้า', 'ค่า', '3'] 
accurancy: 1.0
['ชาบู', 'บริการ', 'คุณภาพ', 'พนักงาน', 'นึก', 'นั่ง'] 
accurancy: 0.4583333333333333
